Import the necessary dependent libraries

In [1]:
import os
from module import MLPModule
from utils import aggregate_weights, save_global_measure, load_client_data, calculate_global_measures
from tqdm import tqdm
import torch
from client import EMClient
from client_selector import ClientSelector

Set the necessary parameters and hyperparameters for the experiment and model

In [2]:
seed = 54
dataset_name = "n-baiot"
selector_type = 'dqn'
num_rounds = 25
input_dim = 0
n_mix_distribute = 2
select_num = 30

if dataset_name == "unsw-nb15":
    input_dim = 47
elif dataset_name == "n-baiot":
    input_dim = 115

if torch.cuda.is_available():
    device = torch.device("cuda")
# elif torch.backends.mps.is_available():
#     device = torch.device("mps:0")
else:
    device = torch.device("cpu")

hyperparameters = {
    'input_dim': input_dim,
    'lr': 0.005,
    'models_num': n_mix_distribute,
    'hidden_neurons_num': 512,
    'batch_size': 128,
    'seed': seed,
    'device': device
} 

select_hyperparameters = {
    'enable': True,
    'is_train': False,
    'type_name': selector_type,
    'lr': 0.001,
    'global_models_num': n_mix_distribute,
    'pca_n_components': 15,
    'buffer_batch_size': 16,
    'buffer_size': 1000,
    'reward_lambda_value': 64,
    'target_accuracy': 0.99,
    'epsilon_start': 0.8,
    'epsilon_end': 0.2,
    'epsilon_decay': 100,
    'gamma': 0.99,
    'model_path': "./dqn_models/17-10-2023_16-04-52_unsw-nb15",
    'seed': seed,
    'device': device
}

In [3]:
client_data_list = load_client_data(os.path.join(dataset_name, "split"), device)

Generate clients

In [4]:
def create_clients(client_data, hyperparameter):
    client_list = []
    for i, (train_set, test_set, val_set) in enumerate(client_data):
        c = EMClient(
            id=i,
            hyperparameters=hyperparameter,
            train_set=train_set,
            test_set=test_set,
            val_set=val_set,
        )
        client_list.append(c)
    return client_list
clients = create_clients(client_data_list, hyperparameters)

Generate client selector

In [5]:
selector = ClientSelector(clients, select_num, select_hyperparameters)

DQN selection of clients
Successfully loaded model


FedEM training process

In [6]:
# Initialize multiple global models for each mixture distribution
global_models = [MLPModule(hyperparameters) for _ in range(n_mix_distribute)]

# Pre-trained
model_weights_lists = [[] for _ in range(n_mix_distribute)]
client_n_samples_list = {}

for client in clients:
    client.receive_global_model(global_models)
    client_models_parameters, client_measures, client_n_samples = client.local_fit_and_upload_parameters()
    client_n_samples_list[client.id] = client_n_samples

    for i in range(n_mix_distribute):
        model_weights_lists[i].append(client_models_parameters[i])

for i in range(n_mix_distribute):
    global_models[i].set_parameters(aggregate_weights(model_weights_lists[i], client_n_samples_list))
    
selector.fit_pca(global_models, model_weights_lists)

In [7]:
# Record training results
record_global_train_measures = [None] * num_rounds
record_global_val_measures = [None] * num_rounds

pbar = tqdm(total=num_rounds)
pbar.reset()
# Perform num_rounds rounds of training
for idx in range(num_rounds):
    client_n_samples_list = {}

    all_client_train_measures = { 'accuracy': {}, 'fpr': {}, 'tpr': {}, 'ber': {}, 'loss': {} }
    all_client_val_measures = { 'accuracy': {}, 'fpr': {}, 'tpr': {}, 'ber': {}, 'loss': {} }

    selected_client = selector.sample_clients(global_models, model_weights_lists)
    model_weights_lists = [[] for _ in range(n_mix_distribute)]

    for client in selected_client:
        # Client receives new global models
        client.receive_global_model(global_models)

        # The client trains the models locally
        client_models_parameters, client_measures, client_n_samples = client.local_fit_and_upload_parameters()
        client_n_samples_list[client.id] = client_n_samples

        for i in range(n_mix_distribute):
            model_weights_lists[i].append(client_models_parameters[i])

        for key in all_client_train_measures:
            all_client_train_measures[key][client.id] = (client_measures['train'][key])
            all_client_val_measures[key][client.id] = (client_measures['val'][key])

    global_train_measures, global_val_measures = calculate_global_measures(selected_client,
                                                                           all_client_train_measures,
                                                                           all_client_val_measures,
                                                                           display_result=True)
    record_global_train_measures[idx] = global_train_measures
    record_global_val_measures[idx] = global_val_measures

    # Aggregate new global models
    for i in range(n_mix_distribute):
        global_models[i].set_parameters(aggregate_weights(model_weights_lists[i], client_n_samples_list))

    selector.update_dqn(global_models, model_weights_lists, global_val_measures['accuracy'])
    pbar.update(1)

  4%|▍         | 1/25 [00:21<08:45, 21.90s/it]

Global Train Set Loss: 0.3564854662808674
Global Train Set Accuracy: 0.8074460676269654
Global Train Set FPR: 0.5821795848608596
Global Train Set TPR: 0.8908942604195896
Global Train Set BER: 0.3456426622206351
Global Validation Set Loss: 0.25452951711527405
Global Validation Set Accuracy: 0.8398555383362363
Global Validation Set FPR: 0.3709891613759337
Global Validation Set TPR: 0.9312680529362287
Global Validation Set BER: 0.18652722088651916


  8%|▊         | 2/25 [00:42<08:10, 21.31s/it]

Global Train Set Loss: 0.22129732094064142
Global Train Set Accuracy: 0.9428741362823874
Global Train Set FPR: 0.34944328064585195
Global Train Set TPR: 0.8691362780749348
Global Train Set BER: 0.24015350128545848
Global Validation Set Loss: 0.09967576938431169
Global Validation Set Accuracy: 0.9952607477672912
Global Validation Set FPR: 0.05776326187657201
Global Validation Set TPR: 0.9643562250510924
Global Validation Set BER: 0.030036851746073158


 12%|█▏        | 3/25 [01:03<07:46, 21.23s/it]

...

Global Train Set Loss: 0.21560443451910913
Global Train Set Accuracy: 0.9464728508122583
Global Train Set FPR: 0.3388939024790989
Global Train Set TPR: 0.8704639327196433
Global Train Set BER: 0.23421498487972767
Global Validation Set Loss: 0.15882652595539376
Global Validation Set Accuracy: 0.9833311383997937
Global Validation Set FPR: 0.03107081687610602
Global Validation Set TPR: 0.9592018157448625
Global Validation Set BER: 0.019267833898955075


 96%|█████████▌| 24/25 [08:20<00:20, 20.71s/it]

Global Train Set Loss: 0.21494612249685738
Global Train Set Accuracy: 0.9464854550972207
Global Train Set FPR: 0.33879513100336217
Global Train Set TPR: 0.870159209107618
Global Train Set BER: 0.23431796094787202
Global Validation Set Loss: 0.1602703723943003
Global Validation Set Accuracy: 0.9858141197690318
Global Validation Set FPR: 0.02641452271092049
Global Validation Set TPR: 0.9591596476185734
Global Validation Set BER: 0.01696077087950692


100%|██████████| 25/25 [08:42<00:00, 20.87s/it]

Global Train Set Loss: 0.21650740987837333
Global Train Set Accuracy: 0.9451772991465828
Global Train Set FPR: 0.3414562242732902
Global Train Set TPR: 0.8705391109380488
Global Train Set BER: 0.23545855666762067
Global Validation Set Loss: 0.15998834145939908
Global Validation Set Accuracy: 0.9829222690265278
Global Validation Set FPR: 0.04704352234344312
Global Validation Set TPR: 0.9590679939481014
Global Validation Set BER: 0.0273210975310042


Save experiment results

In [8]:
suffix_name = ""
if select_num != 100 and selector_type != "random":
    suffix_name = "_dqn_"+str(select_num)
elif select_num != 100: 
    suffix_name = "_random_"+str(select_num)
    
save_global_measure(record_global_train_measures, "train_measures_"+dataset_name+".csv", "FedEM"+suffix_name)
save_global_measure(record_global_val_measures, "val_measures_"+dataset_name+".csv", "FedEM"+suffix_name)

Saved measures to Experimental_results/FedEM_dqn_30/train_measures_n-baiot.csv
Saved measures to Experimental_results/FedEM_dqn_30/val_measures_n-baiot.csv


Save DQN model

In [9]:
selector.save_model(dataset_name)

Test the performance of the model on the unseen test set

In [10]:
all_client_measures = {
    'accuracy': [],
    'fpr': [],
    'tpr': [],
    'ber': [],
    'loss': []
}
client_n_samples_list = {}

# Iterate over clients and evaluate using each global model
for client in clients:
    client_n_samples = 0
    client_measures_sum = {
        'accuracy': 0,
        'fpr': 0,
        'tpr': 0,
        'ber': 0,
        'loss': 0
    }

    # Client receives new global models
    client.receive_global_model(global_models)

    # Evaluate using each global model and aggregate results
    for global_model in global_models:
        client_measures, current_n_samples = client.evaluate_test_set()
        client_n_samples += current_n_samples
        for key in client_measures:
            client_measures_sum[key] += client_measures[key]

    # Average the measures over all models
    for key in client_measures_sum:
        client_measures_sum[key] /= len(global_models)

    client_n_samples_list[client.id] = client_n_samples

    for key in all_client_measures:
        all_client_measures[key].append(client_measures_sum[key])

global_measures = {}
for key in all_client_measures:
    global_measures[key] = sum([all_client_measures[key][c.id] * client_n_samples_list[c.id] for c in clients]) / sum(client_n_samples_list.values())

print("Global Test Set Loss:", global_measures['loss'])
print("Global Test Set Accuracy:", global_measures['accuracy'])
print("Global Test Set FPR:", global_measures['fpr'])
print("Global Test Set TPR:", global_measures['tpr'])
print("Global Test Set BER:", global_measures['ber'])

Global Test Set Loss: 0.07198309101843382
Global Test Set Accuracy: 0.9941475176682385
Global Test Set FPR: 0.026303188812686725
Global Test Set TPR: 0.9864316147533704
Global Test Set BER: 0.014600803740417173
